<a href="https://colab.research.google.com/github/JunSeokCheon/eCommerce_weblog_analysis/blob/master/eCommerce_weblog_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# - 이번 프로젝트는 kaggle에서 제공하는 데이터셋인 eCommerce 웹 로그 데이터 분석 프로젝이다.
### 1. 데이터 전처리
### 2. 데이터 분석 & 시각화

#### 해당 파일은 데이터 전처리 과정이다.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/MyDrive/ecommerce_log_analysis

/content/drive/MyDrive/ecommerce_log_analysis


In [3]:
ls

2019-Nov.csv     ecommerce-behavior-data-from-multi-category-store.zip
2019-Oct.csv     eCommerce_weblog_columns_analysis.ipynb
data_load.ipynb  kaggle.json


## 0. 데이터 load & import

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

%matplotlib inline

In [5]:
ecommerce_df = pd.read_csv("2019-Oct.csv")

In [9]:
ecommerce_df.shape

(42448764, 9)

In [7]:
ecommerce_df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [8]:
ecommerce_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42448764 entries, 0 to 42448763
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   event_time     object 
 1   event_type     object 
 2   product_id     int64  
 3   category_id    int64  
 4   category_code  object 
 5   brand          object 
 6   price          float64
 7   user_id        int64  
 8   user_session   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 2.8+ GB


In [11]:
len(ecommerce_df['brand'].unique())

3445

In [12]:
ecommerce_df[ecommerce_df['product_id'] == 3900821]

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
33666,2019-10-01 03:12:34 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,512448158,52a857d6-b4db-42cb-bf03-b0b66cd36c72
34159,2019-10-01 03:13:12 UTC,purchase,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,512448158,52a857d6-b4db-42cb-bf03-b0b66cd36c72
43664,2019-10-01 03:24:58 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,541410551,50b74e0f-83a4-4114-aaaf-e083225665f6
45540,2019-10-01 03:27:28 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,541410551,50b74e0f-83a4-4114-aaaf-e083225665f6
...,...,...,...,...,...,...,...,...,...
42356920,2019-10-31 18:59:18 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,30.61,565887381,2f7cdf20-52c4-4a40-b6c4-e0c2868d1528
42358341,2019-10-31 19:00:46 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,30.61,565887381,2f7cdf20-52c4-4a40-b6c4-e0c2868d1528
42417421,2019-10-31 20:51:51 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,30.61,555818740,b7799e97-16fb-41cd-90bd-3bde6fc2bcb9
42428127,2019-10-31 21:30:20 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,30.61,542353553,bb42affc-1863-4009-bcc9-bd8ba92fa30a
